# Постановка задачи

Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. 

### Загрузка исходных данных и библиотек

In [140]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [141]:
# %%capture
# !wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [142]:
data = pd.read_excel('data/data_ford_price.xlsx')
data.head(3)

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0


### Обучение линейной регрессии на данных всего датасета (проводится для сравнения с последующими результатами)

In [143]:
# Выделяем из датасета слолбцы с числовыми данными
data = data[['price', 'year', 
             'condition', 
             'cylinders', 'odometer', 'lat', 'long', 'weather']]
# избавляемся от пропусков
data.dropna(inplace = True)
# выделяем целевую переменную 
y = data['price']
# поскольку в задании указано, что нужно провести ТОЛЬКО отбор признаков (без предварительной обработки), 
# формируем матрицу наблюдений только из числовых признаков
x = data.drop(columns='price')
# формируем обучаемую и тестовую выбоки в соотношении 70/30 и задаваем параметр random_state, равный 40.
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Обучаем модель, делаем предсказание и вычисляем метрики, оценивая базовое качество модели 

In [144]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
r2 = metrics.r2_score(y_test, y_predicted)
print('Test R^2: %.3f' % r2)

Test R^2: 0.619


### Выбор метрики для сравнения качества моделей

Для сравнения моделей в качестве метрики выбираем коэффициент детерминации (R^2), поскольку он позволяет сравнивать модели обученные на разных признаках

### Отбор важных признаков методом RFE

Выделяем три наиболее важных признака эти методом

In [145]:
from sklearn.feature_selection import RFE
 
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'cylinders'], dtype=object)

### Отбор важных признаков методом SelectKBest

Выделяем три наиболее важных признака другим методом, SelectKBest

In [146]:
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(f_regression, k=3)

# from sklearn.feature_selection import SelectKBest, f_classif
# selector = SelectKBest(f_classif, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'odometer'], dtype=object)

 ### Моделирование, обучение модели и оценка ее качества на признаках, полученных RFE

Формируем матрицу наблюдений из выделенных наиболее важных признаков 

In [147]:
x = data.drop(columns = ['price', 'odometer', 'lat', 'long', 'weather'])
x.head(3)

,year,condition,cylinders
0,2016,4,6
1,2009,2,8
2,2002,2,8


In [148]:
# Формируем обучаемую и тестовую выборки 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Обучаем модель на признаках, выделенных RFE, делаем предсказание и вычисляем метрики для оценки качества модели

In [149]:
from sklearn import metrics #метрики
model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_predicted = model.predict(X_test)
 
r2 = metrics.r2_score(y_test, y_predicted)
print('Test R^2: %.3f' % r2)

Test R^2: 0.589


### Моделирование, обучение модели и оценка ее качества на признаках, полученных SelectKBest

Формируем матрицу наблюдений на основе выборки SelectKBest

In [150]:
# Формируем обучаемую и тестовую выборки 
x = data.drop(columns = ['price', 'cylinders', 'lat', 'long', 'weather'])
x.head(3)

,year,condition,odometer
0,2016,4,43500
1,2009,2,98131
2,2002,2,201803


In [151]:
# Формируем обучаемую и тестовую выборки 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

Обучаем модель на признаках, выделенных SelectKBest, делаем предсказание и вычисляем метрики для оценки качества модели

In [152]:
from sklearn import metrics #метрики
model = LinearRegression()
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_predicted = model.predict(X_test)

r2 = metrics.r2_score(y_test, y_predicted)
print('Test R^2: %.3f' % r2)

Test R^2: 0.598


**Вывод:** Обе модели показали удовлетворительные результаты (метрика R^2 у обеих выше 0.5), но **модель на признаках, отобранных методом SelectKBest, показала несколько лучшие результаты.**